In [ ]:
!pip install crewai openai langchain-openai

import os
import logging
from getpass import getpass
from typing import List, Dict
from crewai import Agent, Task, Crew
from langchain_openai import ChatOpenAI

os.environ["OPENAI_API_KEY"] = "Put your key here"

logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

llm = ChatOpenAI(model="gpt-4o-mini", temperature=0.2)


def create_unbiased_answer_agent() -> Agent:
    """
    Creates an agent specialized in generating initial answers with minimized biases.
    This agent is an expert in presenting factual, clear, and equitable explanations
    for middle-school audiences (ages 11-14), covering a wide range of bias categories.
    """
    return Agent(
        role="Unbiased Answer Agent",
        goal=(
            "Provide a thorough, accurate answer to the user's question, ensuring that "
            "all potential biases—gender, racial, cultural, socioeconomic, disability, ageism, "
            "automation bias, and any other known bias—are identified and mitigated in your response."
        ),
        backstory=(
            "You are a globally recognized educator and AI ethics specialist with deep "
            "expertise in writing for pre-teens. Your mission is to deliver perfectly neutral, "
            "well-structured, and engaging answers that cater to diverse backgrounds."
        ),
        allow_delegation=False,
        verbose=True,
        llm=llm
    )


def create_fairness_enforcer_agent() -> Agent:
    """
    Creates an agent that refines initial answers by enhancing inclusivity,
    diverse representation, and critical-thinking prompts.
    """
    return Agent(
        role="Fairness Improvement Agent",
        goal=(
            "Review the initial answer and rewrite it to ensure exemplary inclusive language, "
            "balanced cultural representation, sensitivity to disabilities and socioeconomic contexts, "
            "and prompts that encourage students to think independently."
            "Don't consider incusivity and communities in unrelated topics, like mathematics and so on."
            "Also includes famous things from first response because children should know that as well."
        ),
        backstory=(
            "You are an equity-focused curriculum designer and educational psychologist. "
            "You specialize in adapting content to reflect multiple identities and learning styles "
            "while empowering students to ask questions."
        ),
        allow_delegation=False,
        verbose=True,
        llm=llm
    )


def create_context_validator_agent() -> Agent:
    """
    Creates an agent that verifies age-appropriateness,
    alignment with educational standards, and clarity for 11-14-year-olds.
    """
    return Agent(
        role="Educational Context Validator",
        goal=(
            "Ensure the refined answer uses vocabulary, examples, and instructional scaffolding "
            "appropriate for middle school learners, aligns with common curriculum goals, and "
            "encourages problem-solving skills rather than passive consumption."
        ),
        backstory=(
            "You are a seasoned middle-school teacher and curriculum specialist experienced in "
            "designing lesson materials that promote deep understanding and student autonomy."
        ),
        allow_delegation=False,
        verbose=True,
        llm=llm
    )


def create_tasks(question: str) -> List[Task]:
    """
    Given a user question, create a sequence of tasks:
    1. Generate an initial unbiased answer.
    2. Refine for fairness and inclusivity.
    3. Validate educational context and engagement.

    Returns:
        List[Task]: Ordered tasks to run through the Crew.
    """
    logger.info("Creating tasks for question: %s", question)

    answer_agent = create_unbiased_answer_agent()
    fairness_agent = create_fairness_enforcer_agent()
    validator_agent = create_context_validator_agent()

    task1 = Task(
        description=(
            f"You are provided with the user question below. Provide a comprehensive, unbiased, "
            f"and bias-free answer suitable for ages 11-14. Identify and mitigate all biases in your text.\n\n"
            f"User Question: '{question}'"
        ),
        expected_output="Initial unbiased answer in clear, age-appropriate language.",
        agent=answer_agent
    )

    task2 = Task(
        description=(
            "Review the answer from Task 1. Enhance it by incorporating inclusive examples, "
            "Add more half the answer given by the previous agent, and add underrated things in it. Let famous things be there. Don't trade those for more inclusivity."
            "representative scenarios covering diverse cultures, genders, abilities, and socioeconomic "
            "backgrounds. Ensure that the content prompts critical thinking rather than rote responses."
            "If asked 1 or 2 number of best things, you can give more number of things back to make it fair and evenly distributed amongnst all things possible."
            "Add a question or assignment at the end."
            "If there are more famous things which have been asked, along with that add underrated ones as well."
            "If answer is skewed to one specific region, then distribute it around world along different continents."
            "Don't spoon feed, and make students learn by asking mindful questions and recommend reading and at the end give some shocking and interesting facts."
        ),
        expected_output="Refined answer with balanced representation and thought-provoking elements.",
        agent=fairness_agent
    )

    task3 = Task(
        description=(
            "Evaluate the refined answer from Task 2 for developmental appropriateness. "
            "Confirm vocabulary, sentence complexity, and pedagogical techniques align with"
            "common middle school standards."
        ),
        expected_output="Final polished answer response.",
        agent=validator_agent
    )

    return [task1, task2, task3]

crew = Crew(
    agents=[
        create_unbiased_answer_agent(),
        create_fairness_enforcer_agent(),
        create_context_validator_agent()
    ],
    tasks=[],
    verbose=True
)


def process_question(question: str) -> Dict:
    """
    Runs the full Crew pipeline on a user question.

    Args:
        question (str): The user's question to answer.

    Returns:
        Dict: The aggregated results from all agents including final answer and notes.
    """
    logger.info("Processing question through Crew: %s", question)
    crew.tasks = create_tasks(question)
    result = crew.kickoff(inputs={})
    logger.info("Processing complete.")
    return result

if __name__ == "__main__":
    sample_question = "Name the best writers."




In [ ]:
!pip install ipywidgets
!jupyter nbextension enable --py widgetsnbextension
import ipywidgets as widgets
from IPython.display import display, clear_output


Enabling notebook extension jupyter-js-widgets/extension...
Paths used for configuration of notebook: 
    	/root/.jupyter/nbconfig/notebook.json
Paths used for configuration of notebook: 
    	
      - Validating: OK
Paths used for configuration of notebook: 
    	/root/.jupyter/nbconfig/notebook.json


In [ ]:

question_input = widgets.Textarea(
    value='',
    placeholder='Enter your question here...',
    description='Question:',
    layout=widgets.Layout(width='100%', height='80px')
)


process_button = widgets.Button(description='Get Answer', button_style='success')

output_area = widgets.Output()

def on_process_button_clicked(b):
    with output_area:
        clear_output()
        print("Processing...")
        question = question_input.value
        if question.strip():
            response = process_question(question)
            print("Final Crew Output:\n", response)
        else:
            print("Please enter a valid question.")

process_button.on_click(on_process_button_clicked)

display(question_input, process_button, output_area)


Textarea(value='', description='Question:', layout=Layout(height='80px', width='100%'), placeholder='Enter you…

Button(button_style='success', description='Get Answer', style=ButtonStyle())

Output()